In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import svm
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.model_selection import train_test_split


In [2]:
#Read Dataset using Pandas
data = pd.read_csv('shuffled-full-set-hashed.csv')

In [3]:
#Create dataframe with column names
df = pd.DataFrame(data.values, columns=['label','doc'])

In [4]:
#Count NA for each column in the dataset
df.isna().sum()

label     0
doc      45
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62203 entries, 0 to 62202
Data columns (total 2 columns):
label    62203 non-null object
doc      62158 non-null object
dtypes: object(2)
memory usage: 972.0+ KB


In [6]:
#Remove Na records from the dataset
df = df.dropna()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62158 entries, 0 to 62202
Data columns (total 2 columns):
label    62158 non-null object
doc      62158 non-null object
dtypes: object(2)
memory usage: 1.4+ MB


In [8]:
df['label'].value_counts()

BILL                       18959
POLICY CHANGE              10616
CANCELLATION NOTICE         9729
BINDER                      8952
DELETION OF INTEREST        4825
REINSTATEMENT NOTICE        4367
DECLARATION                  967
CHANGE ENDORSEMENT           889
RETURNED CHECK               749
EXPIRATION NOTICE            734
NON-RENEWAL NOTICE           624
BILL BINDER                  289
INTENT TO CANCEL NOTICE      229
APPLICATION                  229
Name: label, dtype: int64

In [9]:
#Describe the Dataset
df.describe()

,label,doc
count,62158,62158
unique,14,60175
top,BILL,bf064c332aa1 079935e500e5 1a4dd36c6de0 7efa289...
freq,18959,11


In [10]:
df.describe().transpose()

,count,unique,top,freq
label,62158,14,BILL,18959
doc,62158,60175,bf064c332aa1 079935e500e5 1a4dd36c6de0 7efa289...,11


In [11]:
labels=df.iloc[:,0].values
docs=df.iloc[:,1].values

In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [13]:
le.classes_

array(['APPLICATION', 'BILL', 'BILL BINDER', 'BINDER',
       'CANCELLATION NOTICE', 'CHANGE ENDORSEMENT', 'DECLARATION',
       'DELETION OF INTEREST', 'EXPIRATION NOTICE',
       'INTENT TO CANCEL NOTICE', 'NON-RENEWAL NOTICE', 'POLICY CHANGE',
       'REINSTATEMENT NOTICE', 'RETURNED CHECK'], dtype=object)

In [14]:
tst_size=0.15
vld_size=0.15

X_train_test, X_valid, y_train_test, y_valid = train_test_split(df['doc'], df['label'], test_size = vld_size) 
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=tst_size)


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = TfidfVectorizer()
tfidf_train = vectorizer.fit_transform(X_train)
tfidf_test = vectorizer.transform(X_test)
tfidf_valid = vectorizer.transform(X_valid)


transformer = TfidfTransformer()
tfidf_train_vectors = transformer.fit_transform(tfidf_train)
tfidf_valid_vectors = transformer.fit_transform(tfidf_valid)
tfidf_test_vectors = transformer.fit_transform(tfidf_test)

In [16]:
from sklearn import tree
dtree = tree.DecisionTreeClassifier()
dtree_fit = dtree.fit(tfidf_train_vectors, y_train) 

In [17]:
dtree_pred_val = dtree_fit.predict(tfidf_valid_vectors)
dtree_pred_test = dtree_fit.predict(tfidf_test_vectors)
print(confusion_matrix(y_valid, dtree_pred_val))
print(accuracy_score(y_valid, dtree_pred_val))
print(accuracy_score(y_valid, dtree_pred_val, normalize = False))
print(confusion_matrix(y_test, dtree_pred_test))
print(accuracy_score(y_test, dtree_pred_test))
print(accuracy_score(y_test, dtree_pred_test, normalize = False))

[[  18    7    1    6    2    1    1    1    1    0    0    4    1    0]
 [   1 2287   32   61  156    0   29    4   24   28    2   92   48    5]
 [   1   22    4    0    2    0    0    0    1    0    0    8    0    0]
 [   2   70    3 1058   22    9   43   10    0    0    2  130    6    6]
 [   0  139    1   12 1169    1    2   80    1    6    3   51   41    1]
 [   0    2    0    7    1   99    1    3    1    0    2   26    0    0]
 [   0   33    2   33    9    0   33    3    2    0    2   21    3    0]
 [   0    4    0    1   65    2    7  647    0    0    0   13    4    0]
 [   0   16    0    2    5    0    1    1   69    0    0    4    0    0]
 [   0    7    0    1    4    0    0    1    0   19    0    1    0    0]
 [   0    4    0    0   13    2    1    0    7    0   52    5    1    0]
 [   5  132    5  137   56   21   22   19    9    0    9 1136   20   11]
 [   0   13    0    0   30    0    2    3    3    0    0   20  600    0]
 [   0    7    1    4    1    0    3    1    0    0

In [18]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
lr_fit = lr.fit(tfidf_train_vectors, y_train)

/Users/sumithkumargannarapu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [19]:
lr_pred_val = lr_fit.predict(tfidf_valid_vectors)
lr_pred_test = lr_fit.predict(tfidf_test_vectors)
print(confusion_matrix(y_valid, lr_pred_val))
print(accuracy_score(y_valid, lr_pred_val))
print(accuracy_score(y_valid, lr_pred_val, normalize = False))
print(confusion_matrix(y_test, lr_pred_test))
print(accuracy_score(y_test, lr_pred_test))
print(accuracy_score(y_test, lr_pred_test, normalize = False))

[[   7    5    0   23    0    0    0    0    0    0    0    8    0    0]
 [   2 2548    0   57   88    1    0    0    1    1    0   69    2    0]
 [   0   28    1    3    0    0    0    0    0    0    0    6    0    0]
 [   0   44    0 1227    5    1    1    2    0    0    0   81    0    0]
 [   0  131    0   13 1261    0    0   45    1    0    2   38   16    0]
 [   0    2    0    3    4  106    0    0    0    0    0   27    0    0]
 [   0   38    0   68    3    0    4    4    0    0    0   23    1    0]
 [   0    3    0    1   88    1    0  645    0    0    0    5    0    0]
 [   0   31    0    3   18    0    0    1   40    1    0    4    0    0]
 [   0   13    0    1   12    0    0    1    0    4    0    2    0    0]
 [   0    2    0    0   36    0    0    1    0    0   42    4    0    0]
 [   1  123    0  131   38   39    1    5    1    0    0 1231   12    0]
 [   0   13    0    4   43    0    0    7    1    0    0   20  583    0]
 [   0    9    0    5    1    0    0    0    0    0

In [ ]:
from sklearn.svm import SVC
svm = svm.SVC()
svm_fit = svm.fit(tfidf_train_vectors, y_train)

/Users/sumithkumargannarapu/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
svm_pred_val = svm_fit.predict(tfidf_valid_vectors)
svm_pred_test = svm_fit.predict(tfidf_test_vectors)
print(confusion_matrix(y_valid, svm_pred_val))
print(accuracy_score(y_valid, svm_pred_val))
print(accuracy_score(y_valid, svm_pred_val, normalize = False))
print(confusion_matrix(y_test, svm_pred_test))
print(accuracy_score(y_test, svm_pred_test))
print(accuracy_score(y_test, svm_pred_test, normalize = False))

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_fit = rf.fit(tfidf_train_vectors, y_train)

/Users/sumithkumargannarapu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [19]:
rf_pred_val = rf_fit.predict(tfidf_valid_vectors)
rf_pred_test = rf_fit.predict(tfidf_test_vectors)
print(confusion_matrix(y_valid, rf_pred_val))
print(accuracy_score(y_valid, rf_pred_val))
print(accuracy_score(y_valid, rf_pred_val, normalize = False))
print(confusion_matrix(y_test, rf_pred_test))
print(accuracy_score(y_test, rf_pred_test))
print(accuracy_score(y_test, rf_pred_test, normalize = False))

[[  18    8    0    9    1    0    1    0    0    0    0    1    0    0]
 [   1 2588    7   48  111    0    7    1   20    1    0  106    0    1]
 [   0   23    7    5    1    0    0    0    0    0    0    4    0    0]
 [   2   71    0 1121    5    1    3    1    0    0    0  128    3    0]
 [   0  143    0   12 1213    0    2   43    0    4    3   53   25    0]
 [   0    8    0    4    2   86    0    0    0    0    0   20    1    0]
 [   0   28    0   67    8    0   16    4    0    0    0   24    0    1]
 [   0    3    0    3  112    1    1  576    0    0    0   10    2    0]
 [   0   25    0    1   18    0    0    1   57    0    0    5    1    0]
 [   0    7    0    2    9    0    0    4    0    9    0    1    0    0]
 [   0    2    0    2   31    0    2    0    2    1   42    4    2    0]
 [   0  121    1  115   43   21    3    6    2    0    1 1208   15    0]
 [   0   16    0    0   75    0    1    3    2    0    1   21  562    0]
 [   0   11    0    1    2    0    0    0    0    0